In [1]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
import cython
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
import pickle
import skopt

In [2]:
g_ket = basis(5,0)
u_ket = basis(5,1)
x_ket = basis(5,2)
a_ket = basis(5,3)
b_ket = basis(5,4)

g_bra = g_ket.dag()
u_bra = u_ket.dag()
x_bra = x_ket.dag()
a_bra = a_ket.dag()
b_bra = b_ket.dag()

g_id = g_ket*g_bra
u_id = u_ket*u_bra
x_id = x_ket*x_bra
a_id = a_ket*a_bra
b_id = b_ket*b_bra

gx_swap = x_ket*g_bra
ux_swap = x_ket*u_bra

spont_decay = b_ket*x_bra
cav_decay = a_ket*g_bra

In [3]:
H_cav = -(gx_swap + gx_swap.dag())
H_rabi = -1/2*(ux_swap + ux_swap.dag())
H_atom = u_id + g_id

In [4]:
gamma = 2*np.pi
pulse_time = 10/gamma
STEPS_PER_SEC = 1000

H_args = {
    'T': 10/gamma,
    'wSTIRAP': np.pi/pulse_time
    }
H_ops = Options(rhs_reuse=True)

def create_time(pulse_time, kappa):
    num_steps1 = round(STEPS_PER_SEC*pulse_time + 1)
    if num_steps1 < 1001:
        num_steps1 = 1001
        
    t_res = 2
    int_time1 = round(pulse_time, t_res)*10**t_res
    t1 = np.linspace(0, int_time1, num_steps1)
    t1 /= 10**t_res

    if kappa > 1:
        added_time = 1/gamma
    else:
        added_time = gamma/kappa

    num_steps2 = round(STEPS_PER_SEC*added_time + 1)
    if num_steps2 < 1001:
        num_steps2 = 1001
    t_res = 2
    int_time2 = round(pulse_time + added_time, t_res)*10**t_res
    t2 = np.linspace(t1[-1], int_time2, num_steps1)
    t2 /= 10**t_res

    return [t1, t2]

sin_pulse = 'sin'
tophat_pulse = 'tophat'
opts = Options(rhs_reuse=True, nsteps=100000)

In [5]:
def save_obj(object, name):
    with open(name+'.pickle', 'wb') as f:
        print(name+'.pickle')
        pickle.dump(object, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_obj(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [6]:
NUM_GS = 40
NUM_KAPPAS = 30
kappa_min = 0.1
kappa_max = 100
g_min = 1
g_max = 200

kappas = np.logspace(np.log10(kappa_min), np.log10(kappa_max), NUM_KAPPAS)
g_ratios = np.logspace(np.log10(g_min), np.log10(g_max), NUM_GS)
current_kappa = kappas[0]

In [7]:
def find_efficiency(g_ratio, omega, delta, kappa, pulse_time, psi0=u_ket, pulse_shape=sin_pulse):

    omega *= 2*np.pi
    delta *= 2*np.pi
    g0 = g_ratio*gamma

    H_args['omega'], H_args['delta'], H_args['g0'] = omega, delta, g0
    t1, t2 = create_time(pulse_time, kappa)

    global current_kappa
    if kappa != current_kappa:
        opts.rhs_reuse = False
        current_kappa = kappa

    c_ops = [np.sqrt(kappa*gamma/2)*cav_decay, np.sqrt(gamma/2)*spont_decay]

    if pulse_shape =='sin':
        pulse = 'omega*np.sin(wSTIRAP*t)**2'
    else:
        pulse = 'omega'

    H = [[H_cav, 'g0'], [H_rabi, pulse], [H_atom, 'delta']]

    result1 = mesolve(H, psi0, t1, c_ops, options=opts, args=H_args)
    # H_args['omega'] = 0
    # result2 = mesolve(H, result1.states[-1], t2, c_ops, options=opts, args=H_args)
    opts.rhs_reuse = True

    return expect(a_id, result1.states[-1])
    

In [8]:
for i in range(1):

    def iterative_optimiser_2D(inputs):
        
        g_ratio, omega_0, delta_0, kappa= inputs

        omg_res = [1000, 300, 100, 30, 10, 3, 1, 0.3, 0.1]
        delt_res = [1000, 300, 100, 30, 10, 3, 1, 0.3, 0.1]

        initial_eff = find_efficiency(g_ratio, omega_0, delta_0, kappa, pulse_time=pulse_time, psi0=u_ket, pulse_shape=sin_pulse)
    
        def find_eff(omega,delta):
            return find_efficiency(g_ratio, omega, delta, kappa, pulse_time)

        for res in omg_res:
            # print(f'res = {res}')
            

            finished = False
            new_step = 0
            

            while not finished:
                # print(omega_0, delta_0, initial_eff)


                if new_step == 0:
                    temp_effs = [initial_eff, find_eff(omega_0-res, delta_0), find_eff(omega_0 + res, delta_0)]
                elif new_step == 1:
                    temp_effs = [initial_eff, temp_effs[0],find_eff(omega_0+res, delta_0)]
                else:
                    temp_effs = [initial_eff, find_eff(omega_0-res,delta_0), temp_effs[0]]
                
                initial_eff = max(temp_effs)
                if np.argmax(temp_effs) == 0:
                    
                    finished = True
                elif np.argmax(temp_effs) == 1:
                    new_step = -1
                    omega_0 -= res
                else:
                    new_step = 1
                    omega_0 += res

        omega_0 = abs(omega_0)

        for res in delt_res:
            # print(f'res = {res}')
            

            finished = False
            new_step = 0
            

            while not finished:
                # print(omega_0, delta_0, initial_eff)


                if new_step == 0:
                    temp_effs = [initial_eff, find_eff(omega_0, delta_0-res), find_eff(omega_0, delta_0+res)]
                elif new_step == 1:
                    temp_effs = [initial_eff, temp_effs[0],find_eff(omega_0, delta_0+res)]
                else:
                    temp_effs = [initial_eff, find_eff(omega_0,delta_0-res), temp_effs[0]]
                
                initial_eff = max(temp_effs)
                if np.argmax(temp_effs) == 0:
                    
                    finished = True
                elif np.argmax(temp_effs) == 1:
                    new_step = -1
                    delta_0 -= res
                else:
                    new_step = 1
                    delta_0 += res

        delta_0 = abs(delta_0)

        def neg_eff(omg_del):
            return -find_eff(omg_del[0], omg_del[1])
        op_result = skopt.gp_minimize(neg_eff, [[omega_0*0.95, omega_0*1.05], [delta_0*0.95, delta_0*1.05 + 1e-7]],x0=[omega_0, delta_0], n_calls=12)
        omega_0, delta_0 = op_result.x
        initial_eff = -op_result.fun
                        
        return {
            'g_ratio': g_ratio,
            'kappa_ratio': kappa,
            'omega': np.abs(omega_0),
            'delta': np.abs(delta_0),
            'efficiency': initial_eff
        }

In [9]:
total_cores = multiprocessing.cpu_count()
if total_cores > 60:
    num_cores = 60
else:
    num_cores = total_cores

In [10]:
first_time = True
input_args = []
output_list = []
for j in range(len(kappas)):
    input_args.append([[g_ratios[k], 100, 0, kappas[j]] for k in range(len(g_ratios))])

for input_set in input_args:

    if not first_time:
        for k in range(len(g_ratios)):
            input_set[k][1] = processed_list['omega']
            input_set[k][2] = processed_list['delta']

    if __name__ == "__main__":
        inputs = tqdm(input_set)
        processed_list = Parallel(n_jobs=num_cores)(delayed(iterative_optimiser_2D)(i) for i in inputs)
        output_list.append(processed_list)
        save_obj(output_list, '2D sin time variance time = 10gamma^-1')
    first_run = False

 80%|████████  | 32/40 [10:01<04:56, 37.01s/it]

In [ ]:
effs = [[output_list[j][k]['efficiency'] for k in range(len(g_ratios))] for j in range(len(kappas))]


def C(g_ratio, kappa):
    return g_ratio**2/(2*kappa)

bench = [[2*C(g,kappa)/(2*C(g,kappa)+1) for g in g_ratios] for kappa in kappas]

diff_effs = [[effs[j][k] - bench[j][k] for k in range(len(g_ratios))] for j in range(len(kappas))]

errs = [[1 - effs[j][k] for k in range(len(g_ratios))] for j in range(len(kappas))]

C_ratio = [[effs[j][k]/(2*C(g_ratios[k], kappas[j])*(1-effs[j][k])) for k in range(len(g_ratios))] for j in range(len(kappas))]

In [ ]:
NUM_CONTS = 20
X,Y = np.meshgrid(g_ratios, kappas)

fig,[ax1,ax2,ax3] = plt.subplots(1,3)
fig.set_size_inches(18,6)

cp1=ax1.contourf([X,Y], effs, levels=NUM_CONTS)
cb1 = fig.colorbar(cp1)
cb1.set_label('Efficiency')
ax1.set_xlabel('$g_0/\gamma$')
ax1.set_ylabel('$\kappa/\gamma$')
cs1 = ax1.contour([X,Y], diff_effs, [0], 'k')
ax1.clabel(cs1, ['$2C/{2C+1}$'])

cp2 = ax2.contourf([X,Y], errs, levels=NUM_CONTS)
cb2 = fig.colorbar(cp2)
cb2.set_label('$\epsilon$')
ax2.set_xlabel('$g_0/\gamma$')
ax2.set_ylabel('$\kappa/\gamma$')
cs2 = ax2.contour([X,Y], errs, [1e-4], 'w')
ax2.clabel(cs2, ['$\epsilon_{th}$'])

cp3 = ax3.contourf([X,Y], C_ratio, levels=NUM_CONTS)
cb3 = fig.colorbar(cp3)
cb3.set_label('$\tilde C/C$')
ax3.set_xlabel('$g_0/\gamma$')
ax3.set_ylabel('$\kappa/\gamma$')
cs3 = ax3.contour([X,Y], C_ratio, [1], 'k')
ax3.clabel(cs3, ['1'])